<a href="https://colab.research.google.com/github/TaahirBhorat/Zindi-Hack-Yassir/blob/master/Catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import model_selection, preprocessing, metrics
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
train = pd.read_csv('/content/drive/My Drive/data 2.csv')

In [ ]:
train.head()

,Unnamed: 0,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,trip_start_timestamp,trip_end_timestamp,trip_seconds
0,0,42.001571,-87.695013,41.965812,-87.655879,3.60,2020-06-30 23:15:00+00:00,2020-06-30 23:30:00+00:00,845
1,1,41.857184,-87.620335,41.835118,-87.618678,4.00,2020-06-30 21:15:00+00:00,2020-06-30 21:45:00+00:00,1860
2,2,41.878866,-87.625192,41.878866,-87.625192,0.80,2020-06-30 21:00:00+00:00,2020-06-30 21:00:00+00:00,180
3,3,42.001571,-87.695013,41.980264,-87.913625,8.58,2020-06-30 20:45:00+00:00,2020-06-30 21:00:00+00:00,1258
4,4,41.953582,-87.723452,41.986712,-87.663416,9.04,2020-06-30 20:30:00+00:00,2020-06-30 21:15:00+00:00,2325


In [ ]:
train.describe()

,Unnamed: 0,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,trip_seconds
count,4.040078e+06,4.040078e+06,4.040078e+06,4.040078e+06,4.040078e+06,4.040078e+06,4.040078e+06
mean,2.020038e+06,4.190048e+01,-8.765669e+01,4.190113e+01,-8.765418e+01,3.812768e+00,8.362724e+02
std,1.166270e+06,3.754009e-02,6.875958e-02,3.854565e-02,5.892418e-02,1.179981e+01,1.052779e+03
min,0.000000e+00,4.166014e+01,-8.791362e+01,4.166014e+01,-8.791362e+01,1.000000e-02,1.000000e+00
25%,1.010019e+06,4.188099e+01,-8.765430e+01,4.188099e+01,-8.765600e+01,9.000000e-01,3.620000e+02
50%,2.020038e+06,4.189207e+01,-8.763275e+01,4.189251e+01,-8.763331e+01,1.600000e+00,6.000000e+02
75%,3.030058e+06,4.190749e+01,-8.762621e+01,4.192102e+01,-8.762621e+01,3.600000e+00,9.600000e+02
max,4.040077e+06,4.202122e+01,-8.753490e+01,4.202122e+01,-8.753139e+01,3.353100e+03,8.638800e+04


In [ ]:
train.isnull().sum()

Unnamed: 0              0
pickup_latitude         0
pickup_longitude        0
dropoff_latitude        0
dropoff_longitude       0
trip_miles              0
trip_start_timestamp    0
trip_end_timestamp      0
trip_seconds            0
dtype: int64

In [ ]:
train.shape[0]

4040078

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

arr = np.vstack((train[['pickup_latitude', 'pickup_longitude']].values,
                    train[['dropoff_latitude', 'dropoff_longitude']].values))
sample_ind = np.random.permutation(len(arr))
kmeans =MiniBatchKMeans(n_clusters=90, batch_size=10000).fit(coords[sample_ind])

In [ ]:
train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['pickup_latitude', 'pickup_longitude']])

train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['dropoff_latitude', 'dropoff_longitude']])

In [ ]:
arr = np.vstack((train[['pickup_latitude', 'pickup_longitude']].values,
                    train[['dropoff_latitude', 'dropoff_longitude']].values))

from sklearn.decomposition import PCA
pca = PCA().fit(arr)
train['pickup_pca0'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 0]
train['pickup_pca1'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 1]
train['dropoff_pca0'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
train['dropoff_pca1'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 1]

In [ ]:
StartTime = pd.to_datetime(train['trip_start_timestamp'], infer_datetime_format=True)

In [ ]:
train['is_peak_traffic'] = [1 if (5<i<9 or 15<i<20) else 0 for i in StartTime.dt.hour]
train['Day_in_week'] = StartTime.dt.dayofweek
train['Day_in_year'] = StartTime.dt.dayofyear
train['Month'] = StartTime.dt.month
train = train.drop('trip_start_timestamp', axis=1)
train = train.drop('trip_end_timestamp', axis=1)


In [ ]:
train.head()

,Unnamed: 0,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,trip_start_timestamp,trip_end_timestamp,trip_seconds,pickup_cluster,dropoff_cluster,pickup_pca0,pickup_pca1,dropoff_pca0,fropoff_pca1,dropoff_pca1,is_peak_traffic,Day_in_week,Day_in_year,Month
0,0,42.001571,-87.695013,41.965812,-87.655879,3.60,2020-06-30 23:15:00+00:00,2020-06-30 23:30:00+00:00,845,14,7,0.074189,-0.078841,0.024569,-0.060186,-0.060186,0,1,182,6
1,1,41.857184,-87.620335,41.835118,-87.618678,4.00,2020-06-30 21:15:00+00:00,2020-06-30 21:45:00+00:00,1860,11,38,-0.048797,0.027455,-0.058535,0.047325,0.047325,0,1,182,6
2,2,41.878866,-87.625192,41.878866,-87.625192,0.80,2020-06-30 21:00:00+00:00,2020-06-30 21:00:00+00:00,180,0,0,-0.036230,0.009131,-0.036230,0.009131,0.009131,0,1,182,6
3,3,42.001571,-87.695013,41.980264,-87.913625,8.58,2020-06-30 20:45:00+00:00,2020-06-30 21:00:00+00:00,1258,14,43,0.074189,-0.078841,0.269228,0.022178,0.022178,0,1,182,6
4,4,41.953582,-87.723452,41.986712,-87.663416,9.04,2020-06-30 20:30:00+00:00,2020-06-30 21:15:00+00:00,2325,37,29,0.082759,-0.023720,0.039334,-0.076788,-0.076788,0,1,182,6


,Unnamed: 0,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,trip_start_timestamp,trip_end_timestamp,trip_seconds,pickup_cluster,dropoff_cluster,pickup_pca0,pickup_pca1,dropoff_pca0,fropoff_pca1,dropoff_pca1,is_peak_traffic,Day_in_week
88,88,41.899602,-87.633308,41.922686,-87.649489,2.40,2020-06-29 23:15:00+00:00,2020-06-29 23:30:00+00:00,743,27,5,-0.020990,-0.007105,0.002611,-0.022523,-0.022523,0,0
89,89,41.986712,-87.663416,41.901207,-87.676356,7.56,2020-06-29 23:15:00+00:00,2020-06-29 23:30:00+00:00,1658,29,42,0.039334,-0.076788,0.019572,0.007402,0.007402,0,0
90,90,41.980264,-87.913625,41.954028,-87.763399,0.60,2020-06-29 22:30:00+00:00,2020-06-29 22:45:00+00:00,1020,43,18,0.269228,0.022178,0.120010,-0.009289,-0.009289,0,0
91,91,41.812949,-87.617860,41.740206,-87.615970,6.34,2020-06-29 22:15:00+00:00,2020-06-29 22:30:00+00:00,1132,82,46,-0.067533,0.067602,-0.096320,0.134434,0.134434,0,0
92,92,41.792592,-87.769615,41.857184,-87.620335,9.49,2020-06-29 21:15:00+00:00,2020-06-29 21:30:00+00:00,1121,34,11,0.065790,0.142895,-0.048797,0.027455,0.027455,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4036258,4036258,41.874005,-87.663518,41.899602,-87.633308,0.10,2013-01-07 00:15:00+00:00,2013-01-07 00:15:00+00:00,600,17,27,-0.002456,0.027885,-0.020990,-0.007105,-0.007105,0,0
4036259,4036259,41.944227,-87.655998,41.986712,-87.663416,4.10,2013-01-07 00:00:00+00:00,2013-01-07 00:15:00+00:00,540,13,29,0.016659,-0.040102,0.039334,-0.076788,-0.076788,0,0
4036260,4036260,41.901207,-87.676356,41.901207,-87.676356,3.60,2013-01-07 00:00:00+00:00,2013-01-07 00:15:00+00:00,780,42,42,0.019572,0.007402,0.019572,0.007402,0.007402,0,0
4036261,4036261,41.899602,-87.633308,41.874005,-87.663518,2.10,2013-01-07 00:00:00+00:00,2013-01-07 00:15:00+00:00,540,27,17,-0.020990,-0.007105,-0.002456,0.027885,0.027885,0,0
